Random circuit to test mapping pass

This is a circuit that requires a connectivity map where the distance between the qubits increases exponentially with depth

In [1]:
from qiskit import QuantumCircuit

import time
import numpy as np

Define QCNN circuit

In [2]:
num_qubits = 20
num_gates = 50

qc=QuantumCircuit(num_qubits)
np.random.seed(123)
 
for i_gate in range(num_gates):
    # Randomly select any two qubits from the list of qubits
    #Set seed
    
    q1, q2 = np.random.choice(num_qubits, 2, replace=False)

    qc.rxx(0.45, q1, q2)
    qc.rz(0.221, q1)
    qc.rz(0.742, q2)

Optionally print out the circuit

In [3]:
# qc.draw()

Define backend

In [4]:
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit_ibm_runtime.fake_provider import FakeTorino

# backend = GenericBackendV2(num_qubits=num_qubits, basis_gates = ['rz', 'rx', 'ry', 'cx'])
target_device = FakeTorino()

Compile with default passes

In [5]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
default_pm = generate_preset_pass_manager(backend=target_device, optimization_level=3)

t1 = time.time()
default_qc = default_pm.run(qc)
t2 = time.time()
print("Time taken: ", t2-t1)

# Get gate counts
gate_counts = default_qc.count_ops()
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0)) 
# print("Number of 1-qubit gates: ", gate_counts.get("rz", 0) + gate_counts.get("rx", 0) + gate_counts.get("ry", 0))

Time taken:  3.347738265991211
OrderedDict({'sx': 619, 'rz': 394, 'cz': 256, 'x': 37})
Number of 2-qubit gates:  256


In [6]:
# print(default_qc)

UCC

In [7]:
#Test with UCC transpiler
import sys
sys.path.append('../')

from ucc import compile

t1 = time.time()
ucc_qc, gate_counts = compile(qc, mode="ucc", get_gate_counts = True, target_device=target_device)
t2 = time.time()
print("Time taken: ", t2-t1)

# # Get gate counts
print(gate_counts)
print("Number of 2-qubit gates: ", gate_counts.get("cx", 0) + gate_counts.get("cz", 0))
print("Number of 1-qubit gates: ", gate_counts.get("rz", 0) + gate_counts.get("rx", 0) + gate_counts.get("ry", 0) + gate_counts.get("h", 0))

Time taken:  0.7301762104034424
OrderedDict({'cx': 280, 'rz': 163, 'rx': 86, 'ry': 78})
Number of 2-qubit gates:  280
Number of 1-qubit gates:  327


In [8]:
# ucc_qc.draw()

Create custom compiler

In [9]:
# from qiskit.transpiler import PassManager
# import qiskit.transpiler.passes as passes 

# custom_pm = PassManager()

# custom_pm.append(passes.Collect2qBlocks())
# custom_pm.append(passes.ConsolidateBlocks())

# # custom_pm.append(passes.Decompose())
# # custom_pm.append(passes.Optimize1qGatesDecomposition())

# custom_qc = custom_pm.run(qc)

# gate_counts = custom_qc.count_ops()
# print(gate_counts)

# custom_qc.draw()

In [10]:
# # Get gate counts
# gate_counts = custom_qc.count_ops()
# print(gate_counts)
# print("Number of 2-qubit gates: ", gate_counts.get("cz", 0) + gate_counts.get("cx", 0))
# print("Number of 1-qubit gates: ", gate_counts.get("sx", 0) + gate_counts.get("rz", 0) + gate_counts.get("x", 0))
